# 🌐 Multi-Language Code Converter

**AI-Powered Code Translation Across 13+ Programming Languages**

A comprehensive code conversion tool that translates code between multiple programming languages using state-of-the-art LLM models. Supports both frontier models (GPT-5, Claude 4.5, Gemini 2.5 Pro, Grok 4) and open-source models (Ollama, Groq, OpenRouter).

## 🎯 Key Features

- **13+ Language Support**: Python, C++, Rust, Java, JavaScript, TypeScript, C, C#, Go, PHP, Swift, Ruby, Kotlin, Julia
- **Multi-LLM Support**: OpenAI, Anthropic, Google, Grok, Ollama, Groq, OpenRouter
- **Performance Optimization**: Generates high-performance code optimized for target language
- **Interactive Interface**: Gradio-based web UI for easy code conversion
- **Compilation Testing**: Automatic compilation and execution for compiled languages (C++, Rust, Go)
- **Quality Analysis**: Code validation and performance metrics

## 📋 Supported Languages

**Source/Target Languages:**
- Python • C++ • Rust • Java • JavaScript • TypeScript • C • C# • Go • PHP • Swift • Ruby • Kotlin • Julia

## 🤖 Supported LLM Models

**Frontier Models:**
- GPT-5 (OpenAI)
- Claude Sonnet 4.5 (Anthropic)
- Gemini 2.5 Pro (Google)
- Grok 4 (xAI)

**Open-Source Models:**
- Qwen2.5 Coder (Ollama)
- DeepSeek Coder v2 (Ollama)
- GPT-OSS 20B (Ollama)
- GPT-OSS 120B (Groq)
- Qwen3 Coder 30B (OpenRouter)

## 1. Setup and Installation

In [ ]:
"""
Multi-Language Code Converter - Imports and Environment Setup

This cell imports required libraries and loads environment variables from the root .env file.
"""

import os
import sys
import subprocess
from pathlib import Path
from typing import Optional, Dict, Tuple, List, Any
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr


def find_root_directory(target_name: str = "llm_engineering") -> Path:
    """
    Find the root directory by traversing up the directory tree.
    
    Args:
        target_name: Name of the target directory to find
        
    Returns:
        Path to the root directory, or current directory if not found
    """
    current_dir = Path.cwd()
    root_dir = current_dir
    
    while root_dir.name != target_name and root_dir.parent != root_dir:
        root_dir = root_dir.parent
    
    return root_dir if root_dir.name == target_name else current_dir


# Load environment variables from root .env file
root_dir = find_root_directory("llm_engineering")
env_path = root_dir / '.env'

if env_path.exists():
    load_dotenv(env_path, override=True)
    print(f"✅ Loaded .env from: {env_path}")
else:
    # Fallback to current directory
    load_dotenv(override=True)
    print(f"⚠️  Root .env not found at {env_path}, using current directory .env")

print("✅ Libraries imported successfully!")

## 2. Environment Setup and API Keys

In [ ]:
"""
Environment Setup and API Key Configuration

Loads and validates API keys from environment variables.
"""

# API Key Configuration
API_KEYS = {
    'openai': os.getenv('OPENAI_API_KEY'),
    'anthropic': os.getenv('ANTHROPIC_API_KEY'),
    'google': os.getenv('GOOGLE_API_KEY'),
    'grok': os.getenv('GROK_API_KEY'),
    'groq': os.getenv('GROQ_API_KEY'),
    'openrouter': os.getenv('OPENROUTER_API_KEY'),
}

# OpenRouter configuration
OPENROUTER_BASE_URL = os.getenv('OPENROUTER_BASE_URL', 'https://openrouter.ai/api/v1')

# API endpoint URLs
API_ENDPOINTS = {
    'anthropic': "https://api.anthropic.com/v1/",
    'gemini': "https://generativelanguage.googleapis.com/v1beta/openai/",
    'grok': "https://api.x.ai/v1",
    'groq': "https://api.groq.com/openai/v1",
    'ollama': "http://localhost:11434/v1",
}


def check_api_key_status(key_name: str, key_value: Optional[str], prefix_length: int = 8) -> None:
    """
    Check and print API key status.
    
    Args:
        key_name: Name of the API key
        key_value: The API key value
        prefix_length: Number of characters to show in prefix
    """
    if key_value:
        prefix = key_value[:prefix_length]
        print(f"✅ {key_name.capitalize()} API Key exists (begins with {prefix})")
    else:
        status = "❌" if key_name == 'openai' else "⚠️"
        optional = "" if key_name == 'openai' else " (optional)"
        print(f"{status} {key_name.capitalize()} API Key not set{optional}")


# Check API key status
print("API Key Status:")
check_api_key_status('openai', API_KEYS['openai'])
check_api_key_status('anthropic', API_KEYS['anthropic'], prefix_length=7)
check_api_key_status('google', API_KEYS['google'], prefix_length=2)
check_api_key_status('grok', API_KEYS['grok'], prefix_length=4)
check_api_key_status('groq', API_KEYS['groq'], prefix_length=4)
check_api_key_status('openrouter', API_KEYS['openrouter'], prefix_length=6)

if API_KEYS['openrouter']:
    print(f"✅ OpenRouter Base URL: {OPENROUTER_BASE_URL}")

## 3. Initialize LLM Clients

In [ ]:
"""
Initialize LLM Clients

Creates OpenAI-compatible clients for various LLM providers.
Priority for OpenAI: OpenRouter > Direct OpenAI API
"""


def initialize_openai_client() -> Optional[OpenAI]:
    """
    Initialize OpenAI client with priority: OpenRouter > Direct OpenAI.
    
    Returns:
        OpenAI client instance or None if no API key is available
    """
    if API_KEYS['openrouter']:
        client = OpenAI(api_key=API_KEYS['openrouter'], base_url=OPENROUTER_BASE_URL)
        print(f"✅ Using OpenAI API via OpenRouter at {OPENROUTER_BASE_URL}")
        return client
    elif API_KEYS['openai']:
        client = OpenAI(api_key=API_KEYS['openai'])
        print("✅ Using direct OpenAI API")
        return client
    else:
        print("⚠️  OpenAI client not available (no API key)")
        return None


def initialize_llm_client(provider: str, api_key: Optional[str], base_url: str) -> Optional[OpenAI]:
    """
    Initialize an LLM client for a specific provider.
    
    Args:
        provider: Name of the provider
        api_key: API key for the provider
        base_url: Base URL for the API
        
    Returns:
        OpenAI client instance or None if API key is not available
    """
    if api_key:
        return OpenAI(api_key=api_key, base_url=base_url)
    return None


# Initialize OpenAI client (with priority handling)
openai_client = initialize_openai_client()

# Initialize other LLM clients
anthropic_client = initialize_llm_client('anthropic', API_KEYS['anthropic'], API_ENDPOINTS['anthropic'])
gemini_client = initialize_llm_client('gemini', API_KEYS['google'], API_ENDPOINTS['gemini'])
grok_client = initialize_llm_client('grok', API_KEYS['grok'], API_ENDPOINTS['grok'])
groq_client = initialize_llm_client('groq', API_KEYS['groq'], API_ENDPOINTS['groq'])
ollama_client = OpenAI(api_key="ollama", base_url=API_ENDPOINTS['ollama'])  # Ollama doesn't require API key
openrouter_client = initialize_llm_client('openrouter', API_KEYS['openrouter'], OPENROUTER_BASE_URL)

print("✅ LLM clients initialized!")

## 4. Model Configuration

In [ ]:
# Available models
FRONTIER_MODELS = {
    "gpt-5": openai_client,
    "claude-sonnet-4-5-20250929": anthropic_client,
    "gemini-2.5-pro": gemini_client,
    "grok-4": grok_client,
}

OPEN_SOURCE_MODELS = {
    "qwen2.5-coder": ollama_client,
    "deepseek-coder-v2": ollama_client,
    "gpt-oss:20b": ollama_client,
    "openai/gpt-oss-120b": groq_client,
    "qwen/qwen3-coder-30b-a3b-instruct": openrouter_client,
}

# Filter out None clients
FRONTIER_MODELS = {k: v for k, v in FRONTIER_MODELS.items() if v is not None}
OPEN_SOURCE_MODELS = {k: v for k, v in OPEN_SOURCE_MODELS.items() if v is not None}

ALL_MODELS = {**FRONTIER_MODELS, **OPEN_SOURCE_MODELS}

print(f"✅ Available models: {list(ALL_MODELS.keys())}")

# Low-cost alternatives (uncomment to use)
# FRONTIER_MODELS = {
#     "gpt-5-nano": openai_client,
#     "claude-haiku-4-5": anthropic_client,
#     "gemini-2.5-flash-lite": gemini_client,
#     "grok-4-fast-non-reasoning": grok_client,
# }

## 5. System Information (for optimization)

In [ ]:
"""
System Information Retrieval

Loads system information for code optimization.
Attempts to use the system_info module from week4 directory, falls back to basic info if unavailable.
"""

import platform


def get_basic_system_info() -> Dict[str, str]:
    """
    Get basic system information using platform module.
    
    Returns:
        Dictionary containing basic system information
    """
    return {
        "os": platform.system(),
        "platform": platform.platform(),
        "processor": platform.processor(),
    }


def load_system_info() -> Dict[str, Any]:
    """
    Load system information, attempting to use system_info module from week4.
    
    Returns:
        Dictionary containing system information
    """
    # Try to find and import system_info from week4 directory
    week4_dir = find_root_directory("week4")
    
    if week4_dir.name == 'week4':
        sys.path.insert(0, str(week4_dir))
        try:
            from system_info import retrieve_system_info
            system_info = retrieve_system_info()
            print("✅ Loaded detailed system information")
            return system_info
        except ImportError:
            print("⚠️  system_info module not found, using basic system info")
            return get_basic_system_info()
    else:
        print("⚠️  week4 directory not found, using basic system info")
        return get_basic_system_info()


# Load system information
system_info = load_system_info()

# Display system information
print("\nSystem Information:")
if isinstance(system_info, dict) and len(system_info) > 3:
    # Detailed system info
    display(Markdown(str(system_info)))
else:
    # Basic system info
    info_text = f"**OS:** {system_info.get('os', 'Unknown')}\n"
    info_text += f"**Platform:** {system_info.get('platform', 'Unknown')}\n"
    info_text += f"**Processor:** {system_info.get('processor', 'Unknown')}"
    display(Markdown(info_text))

## 6. Compilation Commands (for compiled languages)

In [ ]:
"""
Compilation Configuration

Defines compilation and execution commands for compiled languages.
"""

# Compilation commands with optimization flags
COMPILE_COMMANDS: Dict[str, List[str]] = {
    "cpp": [
        "clang++", "-std=c++17", "-Ofast", "-mcpu=native",
        "-flto=thin", "-fvisibility=hidden", "-DNDEBUG",
        "main.cpp", "-o", "main"
    ],
    "rust": ["rustc", "-O", "main.rs", "-o", "main"],
    "go": ["go", "build", "-o", "main", "main.go"],
    "c": [
        "clang", "-std=c11", "-Ofast", "-mcpu=native",
        "-flto=thin", "-DNDEBUG", "main.c", "-o", "main"
    ],
}

# Execution commands
RUN_COMMANDS: Dict[str, List[str]] = {
    "cpp": ["./main"],
    "rust": ["./main"],
    "go": ["./main"],
    "c": ["./main"],
}

# File extensions mapping
FILE_EXTENSIONS: Dict[str, Tuple[str, str]] = {
    "cpp": ("cpp", "main.cpp"),
    "c++": ("cpp", "main.cpp"),
    "rust": ("rs", "main.rs"),
    "go": ("go", "main.go"),
    "c": ("c", "main.c"),
}

print("✅ Compilation commands configured!")

## 7. Core Translation Functions

In [ ]:
"""
Core Translation Functions

Functions for generating prompts and creating message structures for LLM API calls.
"""

# Supported programming languages
SUPPORTED_LANGUAGES: List[str] = [
    "Python", "C++", "Rust", "Java", "JavaScript", "TypeScript",
    "C", "C#", "Go", "PHP", "Swift", "Ruby", "Kotlin", "Julia"
]


def get_system_prompt(source_lang: str, target_lang: str) -> str:
    """
    Generate system prompt for code conversion.
    
    Args:
        source_lang: Source programming language
        target_lang: Target programming language
        
    Returns:
        System prompt string
    """
    return f"""You are an expert code translator specializing in converting {source_lang} code to high-performance {target_lang} code.

Your task is to convert {source_lang} code into optimized {target_lang} code.
- Respond ONLY with {target_lang} code
- Use comments sparingly and only when necessary
- The {target_lang} code must produce identical output to the original {source_lang} code
- Optimize for performance and follow {target_lang} best practices
- Include all necessary imports/headers/packages
- Ensure proper type handling to avoid overflows
- Keep random number generators identical for reproducible results"""


def get_user_prompt(
    source_lang: str,
    target_lang: str,
    source_code: str,
    additional_instructions: str = "",
    system_info_dict: Optional[Dict[str, Any]] = None
) -> str:
    """
    Generate user prompt for code conversion.
    
    Args:
        source_lang: Source programming language
        target_lang: Target programming language
        source_code: Source code to convert
        additional_instructions: Optional additional instructions
        system_info_dict: Optional system information dictionary
        
    Returns:
        User prompt string
    """
    prompt = f"""Convert this {source_lang} code to {target_lang} with the fastest possible implementation that produces identical output.

Requirements:
- Respond ONLY with {target_lang} code
- Do not provide explanations, only code with minimal comments
- Pay attention to number types to ensure no integer overflows
- Include all necessary {target_lang} packages/modules/headers
- Optimize for performance while maintaining correctness

"""
    
    if system_info_dict:
        prompt += f"System information for optimization:\n{system_info_dict}\n\n"
    
    if additional_instructions:
        prompt += f"Additional instructions: {additional_instructions}\n\n"
    
    prompt += f"{source_lang} code to convert:\n\n```{source_lang.lower()}\n{source_code}\n```"
    return prompt


def create_messages(
    source_lang: str,
    target_lang: str,
    source_code: str,
    additional_instructions: str = "",
    system_info_dict: Optional[Dict[str, Any]] = None
) -> List[Dict[str, str]]:
    """
    Create messages structure for LLM API call.
    
    Args:
        source_lang: Source programming language
        target_lang: Target programming language
        source_code: Source code to convert
        additional_instructions: Optional additional instructions
        system_info_dict: Optional system information dictionary
        
    Returns:
        List of message dictionaries for API call
    """
    return [
        {"role": "system", "content": get_system_prompt(source_lang, target_lang)},
        {
            "role": "user",
            "content": get_user_prompt(
                source_lang, target_lang, source_code,
                additional_instructions, system_info_dict
            )
        }
    ]


print("✅ Translation functions defined!")

## 8. Code Conversion Function

In [ ]:
"""
Code Conversion Function

Main function for converting code between programming languages using LLM models.
"""

# Code block markers to remove from LLM responses
CODE_BLOCK_MARKERS = [
    '```cpp', '```rust', '```go', '```java', '```javascript',
    '```typescript', '```python', '```c', '```csharp', '```php',
    '```swift', '```ruby', '```kotlin', '```julia', '```'
]


def clean_code_response(code: str, target_lang: str) -> str:
    """
    Clean markdown code blocks from LLM response.
    
    Args:
        code: Raw code response from LLM
        target_lang: Target language name (for specific marker removal)
        
    Returns:
        Cleaned code string
    """
    cleaned = code
    # Remove language-specific markers
    target_lang_lower = target_lang.lower()
    cleaned = cleaned.replace(f'```{target_lang_lower}', '')
    
    # Remove common code block markers
    for marker in CODE_BLOCK_MARKERS:
        cleaned = cleaned.replace(marker, '')
    
    return cleaned.strip()


def convert_code(
    model_name: str,
    source_lang: str,
    target_lang: str,
    source_code: str,
    additional_instructions: str = "",
    system_info_dict: Optional[Dict[str, Any]] = None
) -> str:
    """
    Convert code from source language to target language using specified model.
    
    Args:
        model_name: Name of the LLM model to use
        source_lang: Source programming language
        target_lang: Target programming language
        source_code: Source code to convert
        additional_instructions: Optional additional instructions
        system_info_dict: Optional system information dictionary
        
    Returns:
        Converted code string or error message
    """
    if model_name not in ALL_MODELS:
        available = list(ALL_MODELS.keys())
        return f"❌ Error: Model '{model_name}' is not available. Available models: {available}"
    
    client = ALL_MODELS[model_name]
    if client is None:
        return f"❌ Error: Client for model '{model_name}' is not initialized. Check API keys."
    
    messages = create_messages(
        source_lang, target_lang, source_code,
        additional_instructions, system_info_dict
    )
    
    try:
        # Use reasoning_effort for GPT models
        reasoning_effort = "high" if "gpt" in model_name.lower() else None
        
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            reasoning_effort=reasoning_effort
        )
        
        converted_code = response.choices[0].message.content
        if not converted_code:
            return "❌ Error: Empty response from model"
        
        # Clean up markdown code blocks
        converted_code = clean_code_response(converted_code, target_lang)
        
        return converted_code
        
    except Exception as e:
        error_msg = str(e)
        return f"❌ Error during conversion: {error_msg}"


print("✅ Code conversion function ready!")

## 9. Compilation and Execution Helpers

In [ ]:
"""
Compilation and Execution Helpers

Functions for compiling and executing code in compiled languages.
"""

# Timeout constants
COMPILE_TIMEOUT = 30  # seconds
EXECUTE_TIMEOUT = 10  # seconds


def compile_and_run(code: str, language: str) -> Tuple[str, bool]:
    """
    Compile and run code for compiled languages.
    
    Args:
        code: Code to compile and run
        language: Programming language name
        
    Returns:
        Tuple of (output, success) where output is stdout or error message,
        and success is a boolean indicating if compilation/execution succeeded
    """
    lang_lower = language.lower()
    
    # Check if compilation is supported
    if lang_lower not in COMPILE_COMMANDS:
        url = f"https://www.programiz.com/{lang_lower}-programming/online-compiler/"
        return (
            f"⚠️  Compilation not supported for {language}. "
            f"Use an online compiler like {url}",
            False
        )
    
    # Get file extension and filename
    file_info = FILE_EXTENSIONS.get(lang_lower)
    if not file_info:
        return f"⚠️  Compilation not configured for {language}", False
    
    _, filename = file_info
    
    try:
        # Write code to file
        with open(filename, "w", encoding="utf-8") as f:
            f.write(code)
        
        # Compile
        compile_cmd = COMPILE_COMMANDS[lang_lower]
        compile_result = subprocess.run(
            compile_cmd,
            check=True,
            text=True,
            capture_output=True,
            timeout=COMPILE_TIMEOUT
        )
        
        # Run
        run_cmd = RUN_COMMANDS[lang_lower]
        run_result = subprocess.run(
            run_cmd,
            check=True,
            text=True,
            capture_output=True,
            timeout=EXECUTE_TIMEOUT
        )
        
        return run_result.stdout, True
        
    except subprocess.TimeoutExpired:
        return "❌ Error: Compilation or execution timed out", False
    except subprocess.CalledProcessError as e:
        error_msg = e.stderr if e.stderr else str(e)
        return f"❌ Compilation/Execution Error: {error_msg}", False
    except OSError as e:
        return f"❌ System Error: {str(e)}", False
    except Exception as e:
        return f"❌ Unexpected Error: {str(e)}", False


print("✅ Compilation helpers ready!")

## 10. Example Python Code (for testing)

In [ ]:
# Example Python code for testing
EXAMPLE_PYTHON_CODE = """import time

def calculate_pi(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations + 1):
        j = i * param1 - param2
        result -= (1 / j)
        j = i * param1 + param2
        result += (1 / j)
    return result

start_time = time.time()
result = calculate_pi(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

print("Example Python code loaded:")
print(EXAMPLE_PYTHON_CODE)

## 11. Gradio Interface

In [ ]:
def convert_with_ui(
    model: str,
    source_lang: str,
    target_lang: str,
    source_code: str,
    additional_instructions: str,
    compile_and_test: bool
) -> Tuple[str, str]:
    """
    Convert code and optionally compile/test it.
    
    Args:
        model: LLM model name
        source_lang: Source programming language
        target_lang: Target programming language
        source_code: Source code to convert
        additional_instructions: Optional additional instructions
        compile_and_test: Whether to compile and test the converted code
        
    Returns:
        Tuple of (converted_code, test_output)
    """
    # Convert code (system_info is available in global scope)
    converted_code = convert_code(
        model, source_lang, target_lang, source_code,
        additional_instructions, system_info
    )
    
    # Compile and test if requested
    test_output = ""
    if compile_and_test and converted_code and not converted_code.startswith("❌"):
        output, success = compile_and_run(converted_code, target_lang)
        test_output = f"**Compilation & Execution Result:**\n\n{output}"
        if not success:
            test_output = f"⚠️ {test_output}"
    
    return converted_code, test_output

"""
Gradio Interface

Creates an interactive web interface for code conversion.
"""

# Create Gradio interface
with gr.Blocks(title="Multi-Language Code Converter", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🌐 Multi-Language Code Converter
    
    Convert code between 13+ programming languages using state-of-the-art LLM models.
    
    **Supported Languages:** Python, C++, Rust, Java, JavaScript, TypeScript, C, C#, Go, PHP, Swift, Ruby, Kotlin, Julia
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            model_dropdown = gr.Dropdown(
                choices=list(ALL_MODELS.keys()),
                value=list(ALL_MODELS.keys())[0] if ALL_MODELS else None,
                label="🤖 Select LLM Model",
                info="Choose the model for code conversion"
            )
            
            source_lang_dropdown = gr.Dropdown(
                choices=SUPPORTED_LANGUAGES,
                value="Python",
                label="📥 Source Language"
            )
            
            target_lang_dropdown = gr.Dropdown(
                choices=SUPPORTED_LANGUAGES,
                value="C++",
                label="📤 Target Language"
            )
            
            compile_checkbox = gr.Checkbox(
                value=False,
                label="🔧 Compile & Test (for compiled languages)",
                info="Automatically compile and run the converted code"
            )
            
            additional_instructions = gr.Textbox(
                label="📝 Additional Instructions (Optional)",
                placeholder="e.g., Use std::vector instead of arrays, optimize for speed...",
                lines=2
            )
        
        with gr.Column(scale=2):
            source_code = gr.Textbox(
                label="📝 Source Code",
                value=EXAMPLE_PYTHON_CODE,
                lines=20
            )
            
            convert_btn = gr.Button("🚀 Convert Code", variant="primary", size="lg")
    
    with gr.Row():
        converted_code = gr.Textbox(
            label="✨ Converted Code",
            lines=20
        )
    
    with gr.Row():
        test_output = gr.Markdown(
            label="🧪 Compilation & Execution Results",
            visible=True
        )
    
    # Event handlers
    convert_btn.click(
        fn=convert_with_ui,
        inputs=[
            model_dropdown,
            source_lang_dropdown,
            target_lang_dropdown,
            source_code,
            additional_instructions,
            compile_checkbox
        ],
        outputs=[converted_code, test_output]
    )

print("✅ Gradio interface created!")
print("Run: demo.launch(inbrowser=True) to start the interface")

In [ ]:
# Launch the Gradio interface
demo.launch(inbrowser=True, share=False)